# Scalable Cloud Data Pipeline for EPL Performance Analysis

## Cloud Assignment - Group Project

### Members: 
### Aditya Pawan Singh - A00050243
### Ritik Rathod - A00049123 

In [1]:
!pip install pyspark
%pip install plotly
import pandas as pd
from pyspark.sql import Window
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import os
import subprocess
from pyspark.sql import SparkSession

import plotly.express as px

Note: you may need to restart the kernel to use updated packages.


In [2]:
S2526 = pd.read_csv(r"C:\Users\Aditya Singh\Desktop\Study\CloudTech\Project\data\E0.csv")
S2425 = pd.read_csv(r"C:\Users\Aditya Singh\Desktop\Study\CloudTech\Project\data\E0 (1).csv")
S2324 = pd.read_csv(r"C:\Users\Aditya Singh\Desktop\Study\CloudTech\Project\data\E0 (2).csv")
S2223 = pd.read_csv(r"C:\Users\Aditya Singh\Desktop\Study\CloudTech\Project\data\E0 (3).csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Aditya Singh\\Desktop\\Study\\CloudTech\\Project\\data\\E0.csv'

In [ ]:

dfs = [S2526, S2425, S2324, S2223]


for df in dfs:
    df.columns = (
        df.columns
        .str.strip()
        .str.replace(" ", "_")
        .str.replace(".", "_")
    )

combined = pd.concat(dfs, ignore_index=True)

print("Rows combined:", len(combined))

combined = combined.drop_duplicates()

combined["Date"] = pd.to_datetime(
    combined["Date"],
    dayfirst=True,
    errors="coerce"
)

combined = combined.sort_values("Date")

# 7. Save the final merged dataset
combined.to_csv(r"C:\Users\Aditya Singh\Desktop\Study\CloudTech\Project\data\SinceCovidPL.csv", index=False)
print("Saved as SinceCovidPL.csv")

combined.head()


Rows combined: 1260
Saved as SinceCovidPL.csv


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,IWA,VCH,VCD,VCA,IWCH,IWCD,IWCA,VCCH,VCCD,VCCA
880,E0,2022-08-05,20:00,Crystal Palace,Arsenal,0,2,A,0,1,...,1.85,4.60,3.5,1.87,4.40,3.55,1.85,4.75,3.5,1.85
881,E0,2022-08-06,12:30,Fulham,Liverpool,2,2,D,1,0,...,1.27,13.00,6.0,1.25,11.00,5.50,1.30,11.50,6.0,1.29
882,E0,2022-08-06,15:00,Bournemouth,Aston Villa,2,0,H,1,0,...,2.05,3.75,3.3,2.00,3.85,3.45,2.00,4.10,3.4,2.00
883,E0,2022-08-06,15:00,Leeds,Wolves,2,1,H,1,1,...,3.15,2.30,3.2,3.10,2.45,3.30,2.95,2.40,3.4,3.00
884,E0,2022-08-06,15:00,Newcastle,Nott'm Forest,2,0,H,0,0,...,5.50,1.62,3.7,5.50,1.63,3.80,6.00,1.57,3.9,7.00


In [ ]:
df = pd.read_csv(r"C:\Users\Aditya Singh\Desktop\Study\CloudTech\Project\data\SinceCovidPL.csv")
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,IWA,VCH,VCD,VCA,IWCH,IWCD,IWCA,VCCH,VCCD,VCCA
0,E0,2022-08-05,20:00,Crystal Palace,Arsenal,0,2,A,0,1,...,1.85,4.60,3.5,1.87,4.40,3.55,1.85,4.75,3.5,1.85
1,E0,2022-08-06,12:30,Fulham,Liverpool,2,2,D,1,0,...,1.27,13.00,6.0,1.25,11.00,5.50,1.30,11.50,6.0,1.29
2,E0,2022-08-06,15:00,Bournemouth,Aston Villa,2,0,H,1,0,...,2.05,3.75,3.3,2.00,3.85,3.45,2.00,4.10,3.4,2.00
3,E0,2022-08-06,15:00,Leeds,Wolves,2,1,H,1,1,...,3.15,2.30,3.2,3.10,2.45,3.30,2.95,2.40,3.4,3.00
4,E0,2022-08-06,15:00,Newcastle,Nott'm Forest,2,0,H,0,0,...,5.50,1.62,3.7,5.50,1.63,3.80,6.00,1.57,3.9,7.00


In [ ]:
df["HomeTeam"].unique()

array(['Crystal Palace', 'Fulham', 'Bournemouth', 'Leeds', 'Newcastle',
       'Tottenham', 'Everton', 'Man United', 'Leicester', 'West Ham',
       'Brighton', 'Aston Villa', 'Brentford', 'Arsenal', 'Southampton',
       'Man City', 'Wolves', 'Chelsea', "Nott'm Forest", 'Liverpool',
       'Burnley', 'Sheffield United', 'Luton', 'Ipswich', 'Sunderland'],
      dtype=object)

In [ ]:
df["AwayTeam"].unique()

array(['Arsenal', 'Liverpool', 'Aston Villa', 'Wolves', "Nott'm Forest",
       'Southampton', 'Chelsea', 'Brighton', 'Brentford', 'Man City',
       'Newcastle', 'Everton', 'Man United', 'Leicester', 'Leeds',
       'Bournemouth', 'Fulham', 'Tottenham', 'West Ham', 'Crystal Palace',
       'Luton', 'Sheffield United', 'Burnley', 'Ipswich', 'Sunderland'],
      dtype=object)

In [ ]:
home_teams = set(df['HomeTeam'].unique())
away_teams = set(df['AwayTeam'].unique())

home_teams == away_teams


True

In [ ]:
print("JAVA_HOME:", os.environ.get("JAVA_HOME"))
print("SPARK_HOME:", os.environ.get("SPARK_HOME"))

# Show Java version
#print(subprocess.check_output(["java", "-version"], stderr=subprocess.STDOUT).decode())


JAVA_HOME: C:\Program Files\Eclipse Adoptium\jdk-17.0.17.10-hotspot
SPARK_HOME: C:\spark-4.0.1-bin-hadoop3


In [ ]:
Keeps = ['Div',
 'Date',
 'Time',
 'HomeTeam',
 'AwayTeam',
 'FTHG',
 'FTAG',
 'FTR',
 'HTHG',
 'HTAG',
 'HTR',
 'Referee',
 'HS',
 'AS',
 'HST',
 'AST',
 'HF',
 'AF',
 'HC',
 'AC',
 'HY',
 'AY',
 'HR',
 'AR']

df_cutoff = df[Keeps]

In [ ]:
spark = SparkSession.builder \
    .appName("FootballAnalysis") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()
spark

In [ ]:
df = spark.read.csv(
    r"C:\Users\Aditya Singh\Desktop\Study\CloudTech\Project\data\SinceCovidPL.csv",
    header=True,
    inferSchema=False
)

df.printSchema()
df.show(5)

root
 |-- Div: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- FTHG: string (nullable = true)
 |-- FTAG: string (nullable = true)
 |-- FTR: string (nullable = true)
 |-- HTHG: string (nullable = true)
 |-- HTAG: string (nullable = true)
 |-- HTR: string (nullable = true)
 |-- Referee: string (nullable = true)
 |-- HS: string (nullable = true)
 |-- AS: string (nullable = true)
 |-- HST: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- HF: string (nullable = true)
 |-- AF: string (nullable = true)
 |-- HC: string (nullable = true)
 |-- AC: string (nullable = true)
 |-- HY: string (nullable = true)
 |-- AY: string (nullable = true)
 |-- HR: string (nullable = true)
 |-- AR: string (nullable = true)
 |-- B365H: string (nullable = true)
 |-- B365D: string (nullable = true)
 |-- B365A: string (nullable = true)
 |-- BFDH: string (nullable = true

In [ ]:
cols = df.columns

df.columns

['Div',
 'Date',
 'Time',
 'HomeTeam',
 'AwayTeam',
 'FTHG',
 'FTAG',
 'FTR',
 'HTHG',
 'HTAG',
 'HTR',
 'Referee',
 'HS',
 'AS',
 'HST',
 'AST',
 'HF',
 'AF',
 'HC',
 'AC',
 'HY',
 'AY',
 'HR',
 'AR',
 'B365H',
 'B365D',
 'B365A',
 'BFDH',
 'BFDD',
 'BFDA',
 'BMGMH',
 'BMGMD',
 'BMGMA',
 'BVH',
 'BVD',
 'BVA',
 'BWH',
 'BWD',
 'BWA',
 'CLH',
 'CLD',
 'CLA',
 'LBH',
 'LBD',
 'LBA',
 'PSH',
 'PSD',
 'PSA',
 'MaxH',
 'MaxD',
 'MaxA',
 'AvgH',
 'AvgD',
 'AvgA',
 'BFEH',
 'BFED',
 'BFEA',
 'B365>2_5',
 'B365<2_5',
 'P>2_5',
 'P<2_5',
 'Max>2_5',
 'Max<2_5',
 'Avg>2_5',
 'Avg<2_5',
 'BFE>2_5',
 'BFE<2_5',
 'AHh',
 'B365AHH',
 'B365AHA',
 'PAHH',
 'PAHA',
 'MaxAHH',
 'MaxAHA',
 'AvgAHH',
 'AvgAHA',
 'BFEAHH',
 'BFEAHA',
 'B365CH',
 'B365CD',
 'B365CA',
 'BFDCH',
 'BFDCD',
 'BFDCA',
 'BMGMCH',
 'BMGMCD',
 'BMGMCA',
 'BVCH',
 'BVCD',
 'BVCA',
 'BWCH',
 'BWCD',
 'BWCA',
 'CLCH',
 'CLCD',
 'CLCA',
 'LBCH',
 'LBCD',
 'LBCA',
 'PSCH',
 'PSCD',
 'PSCA',
 'MaxCH',
 'MaxCD',
 'MaxCA',
 'AvgCH',
 'Avg

In [ ]:
cols = df.columns
ar_index = cols.index("AR")


In [ ]:
keep_cols = cols[:ar_index + 1]
df = df.select(keep_cols)

df.show(5)


+---+----------+-----+--------------+-------------+----+----+---+----+----+---+--------+---+---+---+---+---+---+---+---+---+---+---+---+
|Div|      Date| Time|      HomeTeam|     AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR| Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|
+---+----------+-----+--------------+-------------+----+----+---+----+----+---+--------+---+---+---+---+---+---+---+---+---+---+---+---+
| E0|2022-08-05|20:00|Crystal Palace|      Arsenal|   0|   2|  A|   0|   1|  A|A Taylor| 10| 10|  2|  2| 16| 11|  3|  5|  1|  2|  0|  0|
| E0|2022-08-06|12:30|        Fulham|    Liverpool|   2|   2|  D|   1|   0|  H|A Madley|  9| 11|  3|  4|  7|  9|  4|  4|  2|  0|  0|  0|
| E0|2022-08-06|15:00|   Bournemouth|  Aston Villa|   2|   0|  H|   1|   0|  H|P Bankes|  7| 15|  3|  2| 18| 16|  5|  5|  3|  3|  0|  0|
| E0|2022-08-06|15:00|         Leeds|       Wolves|   2|   1|  H|   1|   1|  D| R Jones| 12| 15|  4|  6| 13|  9|  6|  4|  2|  0|  0|  0|
| E0|2022-08-06|15:00|     Newcastle|Nott

In [ ]:
df = df.withColumn(
    "season_start_year",
    F.when(
        F.col("DATE") >= F.to_date(F.concat(F.year("DATE"), F.lit("-08-01"))),
        F.year("DATE")
    ).otherwise(F.year("DATE") - 1)
).withColumn(
    "Season",
    F.concat(
        F.date_format(F.to_date(F.concat(F.col("season_start_year").cast("string"), F.lit("-01-01"))), "yy"),
        F.lit("/"),
        F.date_format(F.to_date(F.concat((F.col("season_start_year") + 1).cast("string"), F.lit("-01-01"))), "yy")
    )
).drop("season_start_year")


In [ ]:
df.orderBy(F.rand()).show(10, truncate=False)


+---+----------+-----+--------------+-------------+----+----+---+----+----+---+-----------+---+---+---+---+---+---+---+---+---+---+---+---+------+
|Div|Date      |Time |HomeTeam      |AwayTeam     |FTHG|FTAG|FTR|HTHG|HTAG|HTR|Referee    |HS |AS |HST|AST|HF |AF |HC |AC |HY |AY |HR |AR |Season|
+---+----------+-----+--------------+-------------+----+----+---+----+----+---+-----------+---+---+---+---+---+---+---+---+---+---+---+---+------+
|E0 |2024-08-24|15:00|Fulham        |Leicester    |2   |1   |H  |1   |1   |D  |D Bond     |18 |10 |6  |4  |14 |13 |7  |5  |2  |2  |0  |0  |24/25 |
|E0 |2024-03-11|20:00|Chelsea       |Newcastle    |3   |2   |H  |1   |1   |D  |J Brooks   |12 |11 |8  |3  |12 |7  |0  |4  |3  |1  |0  |0  |23/24 |
|E0 |2023-04-01|15:00|Arsenal       |Leeds        |4   |1   |H  |1   |0   |H  |D England  |13 |7  |6  |5  |11 |13 |4  |3  |0  |2  |0  |0  |22/23 |
|E0 |2024-02-11|14:00|West Ham      |Arsenal      |0   |6   |A  |0   |4   |A  |C Pawson   |5  |25 |1  |12 |17 |11 |2  

In [ ]:
df = df.withColumn("DATE", F.to_date("DATE", "yyyy-MM-dd"))

# Create long format: each match appears twice (one per team)
df_long = (
    df.select(
        "DATE", "HomeTeam", "AwayTeam",
        "FTHG", "FTAG", "FTR", "season"
    )
    .withColumn("team", F.col("HomeTeam"))
    .withColumn("opponent", F.col("AwayTeam"))
    .withColumn("is_home", F.lit(1))
    .unionByName(
        df.select(
            "DATE", "HomeTeam", "AwayTeam",
            "FTHG", "FTAG", "FTR", "season"
        )
        .withColumn("team", F.col("AwayTeam"))
        .withColumn("opponent", F.col("HomeTeam"))
        .withColumn("is_home", F.lit(0))
    )
)



In [ ]:
w = Window.partitionBy("team", "season").orderBy("DATE")

df_team_weeks = df_long.withColumn(
    "team_gameweek",
    F.row_number().over(w)
)

home_weeks = df_team_weeks.filter("is_home = 1") \
    .select("DATE", "team", "team_gameweek") \
    .withColumnRenamed("team", "HomeTeam") \
    .withColumnRenamed("team_gameweek", "HomeGW")

away_weeks = df_team_weeks.filter("is_home = 0") \
    .select("DATE", "team", "team_gameweek") \
    .withColumnRenamed("team", "AwayTeam") \
    .withColumnRenamed("team_gameweek", "AwayGW")

df_gw = (
    df.join(home_weeks, ["DATE", "HomeTeam"])
      .join(away_weeks, ["DATE", "AwayTeam"])
      .withColumn("GameWeek", F.greatest("HomeGW", "AwayGW"))
)


In [ ]:
df_gw.orderBy(F.rand()).show(10, truncate=False)

+----------+-------------+-----------+---+-----+----+----+---+----+----+---+-----------+---+---+---+---+---+---+---+---+---+---+---+---+------+------+------+--------+
|DATE      |AwayTeam     |HomeTeam   |Div|Time |FTHG|FTAG|FTR|HTHG|HTAG|HTR|Referee    |HS |AS |HST|AST|HF |AF |HC |AC |HY |AY |HR |AR |Season|HomeGW|AwayGW|GameWeek|
+----------+-------------+-----------+---+-----+----+----+---+----+----+---+-----------+---+---+---+---+---+---+---+---+---+---+---+---+------+------+------+--------+
|2023-02-12|Man United   |Leeds      |E0 |14:00|0   |2   |A  |0   |0   |D  |P Tierney  |16 |11 |6  |5  |10 |13 |7  |3  |2  |1  |0  |0  |22/23 |22    |23    |23      |
|2024-03-02|Wolves       |Newcastle  |E0 |15:00|3   |0   |H  |2   |0   |H  |T Robinson |14 |12 |7  |3  |8  |11 |3  |5  |0  |1  |0  |0  |23/24 |27    |27    |27      |
|2022-08-31|Tottenham    |West Ham   |E0 |19:45|1   |1   |D  |0   |1   |A  |P Bankes   |14 |13 |4  |4  |10 |9  |3  |4  |2  |5  |0  |0  |22/23 |5     |5     |5       

In [ ]:
df = df_gw.withColumn("match_id", F.monotonically_increasing_id())

In [ ]:
df_long_home = df.select(
    "match_id", "season", "gameweek", "date",
    F.col("HomeTeam").alias("team"),
    F.col("AwayTeam").alias("opponent"),
    F.col("FTHG").alias("goals_for"),
    F.col("FTAG").alias("goals_against"),
    F.when(F.col("FTHG") > F.col("FTAG"), "W")
     .when(F.col("FTHG") < F.col("FTAG"), "L")
     .otherwise("D").alias("result")
)

df_long_away = df.select(
    "match_id", "season", "gameweek", "date",
    F.col("AwayTeam").alias("team"),
    F.col("HomeTeam").alias("opponent"),
    F.col("FTAG").alias("goals_for"),
    F.col("FTHG").alias("goals_against"),
    F.when(F.col("FTAG") > F.col("FTHG"), "W")
     .when(F.col("FTAG") < F.col("FTHG"), "L")
     .otherwise("D").alias("result")
)

df_long = df_long_home.union(df_long_away)


In [ ]:
df_long = df_long.withColumn(
    "points",
    F.when(F.col("result") == "W", 3)
     .when(F.col("result") == "D", 1)
     .otherwise(0)
)


In [ ]:
w = Window.partitionBy("season", "team") \
          .orderBy("gameweek") \
          .rowsBetween(Window.unboundedPreceding, Window.currentRow)


In [ ]:
df_table = df_long \
    .withColumn("P", F.count("*").over(w)) \
    .withColumn("W", F.sum(F.when(F.col("result") == "W", 1).otherwise(0)).over(w)) \
    .withColumn("D", F.sum(F.when(F.col("result") == "D", 1).otherwise(0)).over(w)) \
    .withColumn("L", F.sum(F.when(F.col("result") == "L", 1).otherwise(0)).over(w)) \
    .withColumn("GF", F.sum("goals_for").over(w)) \
    .withColumn("GA", F.sum("goals_against").over(w)) \
    .withColumn("GD", F.col("GF") - F.col("GA")) \
    .withColumn("Pts", F.sum("points").over(w))


In [ ]:
season_totals = (
    df_table
    .groupBy("season", "team")
    .agg(
        F.count("*").alias("P"),
        F.sum(F.when(F.col("result") == "W", 1).otherwise(0)).alias("W"),
        F.sum(F.when(F.col("result") == "D", 1).otherwise(0)).alias("D"),
        F.sum(F.when(F.col("result") == "L", 1).otherwise(0)).alias("L"),
        F.sum("goals_for").alias("GF"),
        F.sum("goals_against").alias("GA"),
        (F.sum("goals_for") - F.sum("goals_against")).alias("GD"),
        F.sum("points").alias("Pts"),
    )
)


In [ ]:
w = Window.partitionBy("season").orderBy(
    F.col("Pts").desc(),
    F.col("GD").desc(),
    F.col("GF").desc()
)

season_table = season_totals.withColumn("rank", F.row_number().over(w))



In [ ]:
df_table.orderBy(F.rand()).show(10, truncate=False)

+--------+------+--------+----------+--------------+----------------+---------+-------------+------+------+---+---+---+---+----+----+-----+---+
|match_id|season|gameweek|date      |team          |opponent        |goals_for|goals_against|result|points|P  |W  |D  |L  |GF  |GA  |GD   |Pts|
+--------+------+--------+----------+--------------+----------------+---------+-------------+------+------+---+---+---+---+----+----+-----+---+
|713     |24/25 |4       |2024-09-14|Liverpool     |Nott'm Forest   |0        |1            |L     |0     |4  |3  |0  |1  |7.0 |1.0 |6.0  |9  |
|1193    |25/26 |5       |2025-09-20|Crystal Palace|West Ham        |2        |1            |W     |3     |5  |2  |3  |0  |6.0 |2.0 |4.0  |9  |
|975     |23/24 |24      |2024-02-10|Newcastle     |Nott'm Forest   |3        |2            |W     |3     |24 |11 |3  |10 |51.0|39.0|12.0 |36 |
|322     |23/24 |14      |2023-12-02|Burnley       |Sheffield United|5        |0            |W     |3     |14 |2  |1  |11 |15.0|32.0|-17

In [ ]:
df_table.filter(
    (F.col("season") == "24/25") &
    (F.col("gameweek") == 31)
).orderBy("Pts").show()


+--------+------+--------+----------+--------------+--------------+---------+-------------+------+------+---+---+---+---+----+----+-----+---+
|match_id|season|gameweek|      date|          team|      opponent|goals_for|goals_against|result|points|  P|  W|  D|  L|  GF|  GA|   GD|Pts|
+--------+------+--------+----------+--------------+--------------+---------+-------------+------+------+---+---+---+---+----+----+-----+---+
|    1124| 24/25|      31|2025-04-06|   Southampton|     Tottenham|        1|            3|     L|     0| 31|  2|  4| 25|23.0|74.0|-51.0| 10|
|     670| 24/25|      31|2025-04-07|     Leicester|     Newcastle|        0|            3|     L|     0| 31|  4|  5| 22|25.0|70.0|-45.0| 17|
|     607| 24/25|      31|2025-04-05|       Ipswich|        Wolves|        1|            2|     L|     0| 31|  4|  8| 19|31.0|65.0|-34.0| 20|
|     607| 24/25|      31|2025-04-05|        Wolves|       Ipswich|        2|            1|     W|     3| 31|  9|  5| 17|43.0|59.0|-16.0| 32|
|     

In [ ]:
df_table.orderBy("season", "gameweek", "Pts").show()


+--------+------+--------+----------+--------------+--------------+---------+-------------+------+------+---+---+---+---+---+---+----+---+
|match_id|season|gameweek|      date|          team|      opponent|goals_for|goals_against|result|points|  P|  W|  D|  L| GF| GA|  GD|Pts|
+--------+------+--------+----------+--------------+--------------+---------+-------------+------+------+---+---+---+---+---+---+----+---+
|     126| 22/23|       1|2022-08-06|   Aston Villa|   Bournemouth|        0|            2|     L|     0|  1|  0|  0|  1|0.0|2.0|-2.0|  0|
|     403| 22/23|       1|2022-08-05|Crystal Palace|       Arsenal|        0|            2|     L|     0|  1|  0|  0|  1|0.0|2.0|-2.0|  0|
|     466| 22/23|       1|2022-08-06|       Everton|       Chelsea|        0|            1|     L|     0|  1|  0|  0|  1|0.0|1.0|-1.0|  0|
|    1134| 22/23|       1|2022-08-07|      West Ham|      Man City|        0|            2|     L|     0|  1|  0|  0|  1|0.0|2.0|-2.0|  0|
|     819| 22/23|       1|2

In [ ]:
df_table.printSchema()


root
 |-- match_id: long (nullable = false)
 |-- season: string (nullable = true)
 |-- gameweek: integer (nullable = false)
 |-- date: date (nullable = true)
 |-- team: string (nullable = true)
 |-- opponent: string (nullable = true)
 |-- goals_for: string (nullable = true)
 |-- goals_against: string (nullable = true)
 |-- result: string (nullable = false)
 |-- points: integer (nullable = false)
 |-- P: long (nullable = false)
 |-- W: long (nullable = true)
 |-- D: long (nullable = true)
 |-- L: long (nullable = true)
 |-- GF: double (nullable = true)
 |-- GA: double (nullable = true)
 |-- GD: double (nullable = true)
 |-- Pts: long (nullable = true)



In [ ]:
# Overall Best Teams since Covid.
overall = (
    season_table.groupBy("team")
    .agg(
        F.sum("Pts").alias("points_total"),
        F.sum("GF").alias("goals_for_total"),
        F.sum("GA").alias("goals_against_total"),
        F.sum("GD").alias("goal_difference_total"),
        F.sum("P").alias("games_played"),
        F.count("season").alias("seasons_played")
    )
    .orderBy(F.desc("points_total"))
)

overall.show(25) # There's only been 25 teams since Covid to play in the Prem.



+----------------+------------+---------------+-------------------+---------------------+------------+--------------+
|            team|points_total|goals_for_total|goals_against_total|goal_difference_total|games_played|seasons_played|
+----------------+------------+---------------+-------------------+---------------------+------------+--------------+
|         Arsenal|         276|          272.0|              112.0|                160.0|         126|             4|
|        Man City|         273|          286.0|              121.0|                165.0|         126|             4|
|       Liverpool|         251|          265.0|              149.0|                116.0|         126|             4|
|     Aston Villa|         216|          200.0|              169.0|                 31.0|         126|             4|
|       Newcastle|         212|          234.0|              157.0|                 77.0|         126|             4|
|         Chelsea|         199|          202.0|         

In [ ]:
season_totals = (
    df_table
    .groupBy("season", "team")
    .agg(
        F.count("*").alias("P"),
        F.sum(F.when(F.col("result") == "W", 1).otherwise(0)).alias("W"),
        F.sum(F.when(F.col("result") == "D", 1).otherwise(0)).alias("D"),
        F.sum(F.when(F.col("result") == "L", 1).otherwise(0)).alias("L"),
        F.sum("goals_for").alias("GF"),
        F.sum("goals_against").alias("GA"),
        (F.sum("goals_for") - F.sum("goals_against")).alias("GD"),
        F.sum("points").alias("Pts"),
    )
)


In [ ]:
w = Window.partitionBy("season").orderBy(
    F.col("Pts").desc(),
    F.col("GD").desc(),
    F.col("GF").desc()
)

season_table = season_totals.withColumn("rank", F.row_number().over(w))


In [ ]:
# Proper Season Table
season_table.filter(F.col("season") == "25/26").orderBy("rank").show()


+------+--------------+---+---+---+---+----+----+-----+---+----+
|season|          team|  P|  W|  D|  L|  GF|  GA|   GD|Pts|rank|
+------+--------------+---+---+---+---+----+----+-----+---+----+
| 25/26|       Arsenal| 12|  9|  2|  1|24.0| 6.0| 18.0| 29|   1|
| 25/26|       Chelsea| 12|  7|  2|  3|23.0|11.0| 12.0| 23|   2|
| 25/26|      Man City| 12|  7|  1|  4|24.0|10.0| 14.0| 22|   3|
| 25/26|   Aston Villa| 12|  6|  3|  3|15.0|11.0|  4.0| 21|   4|
| 25/26|Crystal Palace| 12|  5|  5|  2|16.0| 9.0|  7.0| 20|   5|
| 25/26|      Brighton| 12|  5|  4|  3|19.0|16.0|  3.0| 19|   6|
| 25/26|    Sunderland| 12|  5|  4|  3|14.0|11.0|  3.0| 19|   7|
| 25/26|   Bournemouth| 12|  5|  4|  3|19.0|20.0| -1.0| 19|   8|
| 25/26|     Tottenham| 12|  5|  3|  4|20.0|14.0|  6.0| 18|   9|
| 25/26|    Man United| 12|  5|  3|  4|19.0|19.0|  0.0| 18|  10|
| 25/26|       Everton| 12|  5|  3|  4|13.0|13.0|  0.0| 18|  11|
| 25/26|     Liverpool| 12|  6|  0|  6|18.0|20.0| -2.0| 18|  12|
| 25/26|     Brentford| 1

In [ ]:
w_gw = Window.partitionBy("season", "team").orderBy("gameweek")

df_gw = (
    df_table
    .withColumn("P_cum", F.count("*").over(w_gw))
    .withColumn("W_cum", F.sum(F.when(F.col("result") == "W", 1).otherwise(0)).over(w_gw))
    .withColumn("D_cum", F.sum(F.when(F.col("result") == "D", 1).otherwise(0)).over(w_gw))
    .withColumn("L_cum", F.sum(F.when(F.col("result") == "L", 1).otherwise(0)).over(w_gw))
    .withColumn("GF_cum", F.sum("goals_for").over(w_gw))
    .withColumn("GA_cum", F.sum("goals_against").over(w_gw))
    .withColumn("GD_cum", F.col("GF_cum") - F.col("GA_cum"))
    .withColumn("Pts_cum", F.sum("points").over(w_gw))
)


In [ ]:
w_rank = Window.partitionBy("season", "gameweek") \
               .orderBy(F.desc("Pts_cum"), F.desc("GD_cum"), F.desc("GF_cum"))

gameweek_table = df_gw.withColumn("rank", F.row_number().over(w_rank))


In [ ]:
gameweek_table.filter(
    (F.col("season") == "24/25") &
    (F.col("gameweek") == 38)
).orderBy("rank").show()


+--------+------+--------+----------+--------------+--------------+---------+-------------+------+------+---+---+---+---+----+----+-----+---+-----+-----+-----+-----+------+------+------+-------+----+
|match_id|season|gameweek|      date|          team|      opponent|goals_for|goals_against|result|points|  P|  W|  D|  L|  GF|  GA|   GD|Pts|P_cum|W_cum|D_cum|L_cum|GF_cum|GA_cum|GD_cum|Pts_cum|rank|
+--------+------+--------+----------+--------------+--------------+---------+-------------+------+------+---+---+---+---+----+----+-----+---+-----+-----+-----+-----+------+------+------+-------+----+
|     730| 24/25|      38|2025-05-25|     Liverpool|Crystal Palace|        1|            1|     D|     1| 38| 25|  9|  4|86.0|41.0| 45.0| 84|   38|   25|    9|    4|  86.0|  41.0|  45.0|     84|   1|
|    1064| 24/25|      38|2025-05-25|       Arsenal|   Southampton|        2|            1|     W|     3| 38| 20| 14|  4|69.0|34.0| 35.0| 74|   38|   20|   14|    4|  69.0|  34.0|  35.0|     74|   2|


In [ ]:
pdf = gameweek_table.toPandas()

pdf["win_rate"] = pdf["W"] / pdf["gameweek"]

pdf = pdf[(pdf["gameweek"] > 0)]  

fig = px.scatter(
    pdf,
    x="Pts_cum",
    y="win_rate",
    color="team",
    animation_frame="gameweek",
    animation_group="team",
    facet_col="season",  
    hover_data=["team", "gameweek", "points", "win_rate"],
    range_x=[0, 100],
    range_y=[0, 0.9],
    title="Team Performance Progression — Points vs Win Rate per Gameweek",
)

fig.update_layout(
    xaxis_title="Total Points",
    yaxis_title="Win Rate (Wins / Gameweeks)",
    legend_title="Team",
)

fig.show()




In [ ]:
# Select Season from "22/23" or "23/24" or "24/25" or "25/26"
target_season = "24/25"
# Change          ^^^

pdf = (
    gameweek_table
    .filter(F.col("season") == target_season)
    .toPandas()
)

pdf["win_rate"] = pdf["W"] / pdf["gameweek"]
pdf = pdf[pdf["gameweek"] > 0]   

team_colors = {
    "Arsenal": "#EF0107",
    "Man City": "#6CABDD",
    "Liverpool": "#C8102E",
    "Chelsea": "#034694",
    "Man United": "#DA291C",
    "Tottenham": "#132257",
    "Newcastle": "#241F20",
    "Aston Villa": "#670E36",
    "Brighton": "#0057B8",
    "West Ham": "#7A263A",
    "Brentford": "#E30613",
    "Crystal Palace": "#1B458F",
    "Fulham": "#FFFFFF",
    "Wolves": "#FDB913",
    "Everton": "#003399",
    "Nottingham Forest": "#DD0000",
    "Bournemouth": "#DA291C",
    "Leicester": "#003090",
    "Leeds": "#FFCD00",
    "Southampton": "#D71920"
}

default_color = "#888888"

pdf["color"] = pdf["team"].apply(lambda t: team_colors.get(t, default_color))

fig = px.scatter(
    pdf,
    x="Pts_cum",
    y="win_rate",
    color="team",
    animation_frame="gameweek",
    animation_group="team",
    hover_data=["team", "gameweek", "points", "win_rate"],
    range_x=[0, 100],
    range_y=[0, 0.9],
    title=f"Points vs Win Rate Progression ({target_season} Season)",
)

fig.update_layout(
    xaxis_title="Total Points",
    yaxis_title="Win Rate (Wins ÷ Gameweeks)",
    legend_title="Team"
)

fig.show()
